<a href="https://colab.research.google.com/github/abosedealli/LSTM-Multivariate-vs-Univariate/blob/main/LSTM%20NEW%20Untitled5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

!pip install yfinance
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

# load your dataset
# load your dataset
start = "2015-01-01"
data = yf.download('NFLX', start=start)

# create a dataframe with only the open, low, and high prices
data = data[['Open', 'Low', 'High']]
prices = data

# create a variable to store the number of days you want to forecast
forecast_days = 30

# create a variable to store the percentage of data you want to use for testing
test_data_size = 0.2

# create a variable to store the number of days in your dataset
days_in_dataset = len(data)

# create variables to store the number of days you want to use for testing and training
test_days = int(days_in_dataset * test_data_size)
train_days = days_in_dataset - test_days

# create a training dataset
train_data = data.iloc[:train_days]

# create a testing dataset
test_data = data.iloc[train_days:]

# create a scaler to normalize the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# fit the scaler to the training data
scaler.fit(train_data)

# normalize the training data
train_data = scaler.transform(train_data)

# create a dataset with only the open, low, and high prices for the last forecast_days days
X_train = []
y_train = []
for i in range(forecast_days, train_days):
    X_train.append(train_data[i-forecast_days:i])
    y_train.append(train_data[i])
X_train, y_train = np.array(X_train), np.array(y_train)

# reshape the data to be 3D
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 3))

# create a LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 3)))
model.add(LSTM(50))
model.add(Dense(3))

# compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# fit the model to the training data
model.fit(X_train, y_train, epochs=100, batch_size=32)

# normalize the testing data
test_data = scaler.transform(test_data)

# create a dataset with only the open, low, and high prices for the last forecast_days days
X_test = []
y_test = []
for i in range(forecast_days, test_days):
    X_test.append(test_data[i-forecast_days:i])
    y_test.append(test_data[i])
X_test, y_test = np.array(X_test), np.array(y_test)

# reshape the data to be 3D
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 3))

# make predictions
predictions = model.predict(X_test)

# inverse transform the predictions to get the original prices
predictions = scaler.inverse_transform(predictions)
y_test = scaler.inverse_transform(y_test)

# calculate the mean squared error of the predictions
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)
ape = np.abs((y_test - predictions) / y_test)
# Calculate the mean of the absolute percentage difference
mape = np.mean(ape) * 100
print("MAPE: ", mape)



import plotly.graph_objs as go

# generate new data
X_forecast = []
for i in range(forecast_days, forecast_days+30):
    X_forecast.append(prices[i-forecast_days:i])
X_forecast = np.array(X_forecast)


forecast = model.predict(X_forecast)

# Get the last predicted value
last_prediction = predictions[-1]

# Add a dimension to the last value of predictions
last_prediction = np.expand_dims(last_prediction, axis=0)
forecast = scaler.inverse_transform(forecast)
# Insert the last value of predictions at the 0th index of forecast
forecast = np.insert(forecast, 0, last_prediction, axis=0)
trace1 = go.Scatter(x=np.arange(y_test.shape[0]), y=y_test[:,0], name='Actual Open Prices')
trace2 = go.Scatter(x=np.arange(predictions.shape[0]), y=predictions[:,0], name='Predicted Open Prices')
trace3 = go.Scatter(x=np.arange(predictions.shape[0]-1, predictions.shape[0] + forecast.shape[0]-1), y=forecast[:,0], name='Forecasted Open Prices')
trace4 = go.Scatter(x=np.arange(y_test.shape[0]), y=y_test[:,1], name='Actual Low Prices')
trace5 = go.Scatter(x=np.arange(predictions.shape[0]), y=predictions[:,1], name='Predicted Low Prices')
trace6 = go.Scatter(x=np.arange(predictions.shape[0]-1, predictions.shape[0] + forecast.shape[0]-1), y=forecast[:,1], name='Forecasted Low Prices')
trace7 = go.Scatter(x=np.arange(y_test.shape[0]), y=y_test[:,2], name='Actual High Prices')
trace8 = go.Scatter(x=np.arange(predictions.shape[0]), y=predictions[:,2], name='Predicted High Prices')
trace9 = go.Scatter(x=np.arange(predictions.shape[0]-1, predictions.shape[0] + forecast.shape[0]-1), y=forecast[:,2], name='Forecasted High Prices')

data = [trace1, trace2, trace3, trace4, trace5, trace6, trace7, trace8, trace9]
layout = go.Layout(title='Actual, Predicted and Forecasted Prices', xaxis=dict(title='Time'), yaxis=dict(title='Price'))
fig = go.Figure(data=data, layout=layout)
fig.show()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 KB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 KB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 12.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
   

[*********************100%***********************]  1 of 1 completed
Epoch 1/100
50/50 [==============================] - 5s 25ms/step - loss: 0.0310
Epoch 2/100
50/50 [==============================] - 1s 25ms/step - loss: 0.0010
Epoch 3/100
50/50 [==============================] - 1s 24ms/step - loss: 9.2569e-04
Epoch 4/100
50/50 [==============================] - 1s 23ms/step - loss: 9.0215e-04
Epoch 5/100
50/50 [==============================] - 1s 21ms/step - loss: 8.4879e-04
Epoch 6/100
50/50 [==============================] - 1s 25ms/step - loss: 8.3160e-04
Epoch 7/100
50/50 [==============================] - 1s 25ms/step - loss: 8.9920e-04
Epoch 8/100
50/50 [==============================] - 1s 24ms/step - loss: 7.8859e-04
Epoch 9/100
50/50 [==============================] - 1s 23ms/step - loss: 7.3335e-04
Epoch 10/100
50/50 [==============================] - 1s 27ms/step - loss: 7.3551e-04
Epoch 11/100
50/50 [==============================] - 1s 23ms/step - loss: 7.1228e-04
Ep

In [3]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

# load your dataset
# load your dataset
start = "2012-01-01"
data = yf.download('NFLX', start=start)

# create a dataframe with only the open, low, and high prices
data = data[['Open', 'Low', 'High']]
prices = data

# create a variable to store the number of days you want to forecast
forecast_days = 30

# create a variable to store the percentage of data you want to use for testing
test_data_size = 0.2

# create a variable to store the number of days in your dataset
days_in_dataset = len(data)

# create variables to store the number of days you want to use for testing and training
test_days = int(days_in_dataset * test_data_size)
train_days = days_in_dataset - test_days

# create a training dataset
train_data = data.iloc[:train_days]

# create a testing dataset
test_data = data.iloc[train_days:]

# create a scaler to normalize the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# fit the scaler to the training data
scaler.fit(train_data)

# normalize the training data
train_data = scaler.transform(train_data)

# create a dataset with only the open, low, and high prices for the last forecast_days days
X_train = []
y_train = []
for i in range(forecast_days, train_days):
    X_train.append(train_data[i-forecast_days:i])
    y_train.append(train_data[i])
X_train, y_train = np.array(X_train), np.array(y_train)

# reshape the data to be 3D
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 3))

# create a LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 3)))
model.add(LSTM(50))
model.add(Dense(3))

# compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# fit the model to the training data
model.fit(X_train, y_train, epochs=100, batch_size=32)

# normalize the testing data
test_data = scaler.transform(test_data)

# create a dataset with only the open, low, and high prices for the last forecast_days days
X_test = []
y_test = []
for i in range(forecast_days, test_days):
    X_test.append(test_data[i-forecast_days:i])
    y_test.append(test_data[i])
X_test, y_test = np.array(X_test), np.array(y_test)

# reshape the data to be 3D
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 3))

# make predictions
predictions = model.predict(X_test)

# inverse transform the predictions to get the original prices
predictions = scaler.inverse_transform(predictions)
y_test = scaler.inverse_transform(y_test)

# calculate the mean squared error of the predictions
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)
ape = np.abs((y_test - predictions) / y_test)
# Calculate the mean of the absolute percentage difference
mape = np.mean(ape) * 100
print("MAPE: ", mape)



import plotly.graph_objs as go

# generate new data
X_forecast = []
for i in range(forecast_days, forecast_days+30):
    X_forecast.append(prices[i-forecast_days:i])
X_forecast = np.array(X_forecast)


forecast = model.predict(X_forecast)

# Get the last predicted value
last_prediction = predictions[-1]

# Add a dimension to the last value of predictions
last_prediction = np.expand_dims(last_prediction, axis=0)
forecast = scaler.inverse_transform(forecast)
# Insert the last value of predictions at the 0th index of forecast
forecast = np.insert(forecast, 0, last_prediction, axis=0)
trace1 = go.Scatter(x=np.arange(y_test.shape[0]), y=y_test[:,0], name='Actual Open Prices')
trace2 = go.Scatter(x=np.arange(predictions.shape[0]), y=predictions[:,0], name='Predicted Open Prices')
trace3 = go.Scatter(x=np.arange(predictions.shape[0]-1, predictions.shape[0] + forecast.shape[0]-1), y=forecast[:,0], name='Forecasted Open Prices')
trace4 = go.Scatter(x=np.arange(y_test.shape[0]), y=y_test[:,1], name='Actual Low Prices')
trace5 = go.Scatter(x=np.arange(predictions.shape[0]), y=predictions[:,1], name='Predicted Low Prices')
trace6 = go.Scatter(x=np.arange(predictions.shape[0]-1, predictions.shape[0] + forecast.shape[0]-1), y=forecast[:,1], name='Forecasted Low Prices')
trace7 = go.Scatter(x=np.arange(y_test.shape[0]), y=y_test[:,2], name='Actual High Prices')
trace8 = go.Scatter(x=np.arange(predictions.shape[0]), y=predictions[:,2], name='Predicted High Prices')
trace9 = go.Scatter(x=np.arange(predictions.shape[0]-1, predictions.shape[0] + forecast.shape[0]-1), y=forecast[:,2], name='Forecasted High Prices')

data = [trace1, trace2, trace3, trace4, trace5, trace6, trace7, trace8, trace9]
layout = go.Layout(title='Actual, Predicted and Forecasted Prices', xaxis=dict(title='Time'), yaxis=dict(title='Price'))
fig = go.Figure(data=data, layout=layout)
fig.show()


[*********************100%***********************]  1 of 1 completed
Epoch 1/100
69/69 [==============================] - 7s 52ms/step - loss: 0.0132
Epoch 2/100
69/69 [==============================] - 3s 49ms/step - loss: 6.4886e-04
Epoch 3/100
69/69 [==============================] - 2s 27ms/step - loss: 6.2049e-04
Epoch 4/100
69/69 [==============================] - 2s 27ms/step - loss: 5.8893e-04
Epoch 5/100
69/69 [==============================] - 2s 27ms/step - loss: 5.7790e-04
Epoch 6/100
69/69 [==============================] - 2s 25ms/step - loss: 5.5016e-04
Epoch 7/100
69/69 [==============================] - 2s 23ms/step - loss: 5.3902e-04
Epoch 8/100
69/69 [==============================] - 2s 24ms/step - loss: 5.1342e-04
Epoch 9/100
69/69 [==============================] - 2s 23ms/step - loss: 5.0545e-04
Epoch 10/100
69/69 [==============================] - 2s 22ms/step - loss: 4.4827e-04
Epoch 11/100
69/69 [==============================] - 2s 25ms/step - loss: 4.6864e-0

In [4]:
trace1 = go.Scatter(x=np.arange(y_test.shape[0]), y=y_test[:,0], name='Actual Open Prices')
trace2 = go.Scatter(x=np.arange(predictions.shape[0]), y=predictions[:,0], name='Predicted Open Prices')
trace3 = go.Scatter(x=np.arange(predictions.shape[0]-1, predictions.shape[0] + forecast.shape[0]-1), y=forecast[:,0], name='Forecasted Open Prices')
trace4 = go.Scatter(x=np.arange(y_test.shape[0]), y=y_test[:,1], name='Actual Low Prices')
trace5 = go.Scatter(x=np.arange(predictions.shape[0]), y=predictions[:,1], name='Predicted Low Prices')
trace6 = go.Scatter(x=np.arange(predictions.shape[0]-1, predictions.shape[0] + forecast.shape[0]-1), y=forecast[:,1], name='Forecasted Low Prices')
trace7 = go.Scatter(x=np.arange(y_test.shape[0]), y=y_test[:,2], name='Actual High Prices')
trace8 = go.Scatter(x=np.arange(predictions.shape[0]), y=predictions[:,2], name='Predicted High Prices')
trace9 = go.Scatter(x=np.arange(predictions.shape[0]-1, predictions.shape[0] + forecast.shape[0]-1), y=forecast[:,2], name='Forecasted High Prices')

data = [trace1, trace2, trace3, trace4, trace5, trace6, trace7, trace8, trace9]
layout = go.Layout(title='Actual, Predicted and Forecasted Prices', xaxis=dict(title='Time'), yaxis=dict(title='Price'))
fig = go.Figure(data=data, layout=layout)

fig.update_layout(
    title="Netflix Price Prediction",
    xaxis_title="Days",
    yaxis_title="Price",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="Black"
    )
)